<a href="https://colab.research.google.com/github/nastuaah/telegram_bot/blob/master/%D0%9F%D1%80%D0%BE%D0%B5%D0%BA%D1%82.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
!pip3 install pytelegrambotapi
!pip3 install requests
!pip3 install io
!pip install transformers
!pip install validators

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement io (from versions: none)
ERROR: No matching distribution found for io
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=9661ec5c2849a7d21995d015674c37fa408464fedd3c72de1a70521254392357
  Stored in directory: /root/.cache/pip/wheels/f2/ed/dd/d3a556ad245ef9dc570c6bcd2f22886d17b0b408dd3bbb9ac3
Successfully built validators


In [ ]:
import telebot
import requests
import validators
from io import BytesIO
import torch
from PIL import Image
from transformers import ViTFeatureExtractor, AutoTokenizer, VisionEncoderDecoderModel


loc = "ydshieh/vit-gpt2-coco-en"

feature_extractor = ViTFeatureExtractor.from_pretrained(loc)
tokenizer = AutoTokenizer.from_pretrained(loc)
model = VisionEncoderDecoderModel.from_pretrained(loc)
model.eval()

bot = telebot.TeleBot("6219426476:AAEsx4VPKBpEwZ3ExJF2NyZbsvYDJcjjlm8")

@bot.message_handler(commands=['start'])
def start(message): 
  mess = f'Привет, <b>{message.from_user.first_name} </b>' 
  bot.send_message(message.chat.id, mess,parse_mode='html')

@bot.message_handler(commands=['photo'])
def start(message): 
  mess = f'Message description, <b>{message.from_handle_photo} </b>' 
  bot.send_message(message.chat.id, 'Пришлите фотографию')

@bot.message_handler(commands=['text'])
def start(message): 
  mess = f'Message description, <b>{message.from_handle_text} </b>' 
  bot.send_message(message.chat.id, 'Пришлите ссылку на фотографию')

def predict(image):

    pixel_values = feature_extractor(images=image, return_tensors="pt").pixel_values

    with torch.no_grad():
        output_ids = model.generate(pixel_values, max_length=16, num_beams=4, return_dict_in_generate=True).sequences

    preds = tokenizer.batch_decode(output_ids, skip_special_tokens=True)
    preds = [pred.strip() for pred in preds]

    return preds


@bot.message_handler(content_types=['photo'])
def handle_photo(message):
    file_id = message.photo[-1].file_id
    file_url = bot.get_file_url(file_id)

    with Image.open(requests.get(file_url, stream=True).raw) as image:
        preds = predict(image)
        bot.send_message(message.chat.id, preds)

@bot.message_handler(content_types=['text'])
def handle_url(message):

    if not validators.url(message.text):
        bot.send_message(message.chat.id,"Скиньте ссылку!")
        return

    with Image.open(requests.get(message.text, stream=True).raw) as image:
        preds = predict(image)
        bot.send_message(message.chat.id, preds)

bot.infinity_polling()

2023-06-05 14:20:05,623 (__init__.py:960 MainThread) ERROR - TeleBot: "Infinity polling exception: 'Message' object has no attribute 'from_handle_photo'"
ERROR:TeleBot:Infinity polling exception: 'Message' object has no attribute 'from_handle_photo'
2023-06-05 14:20:05,627 (__init__.py:962 MainThread) ERROR - TeleBot: "Exception traceback:
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/telebot/__init__.py", line 955, in infinity_polling
    self.polling(non_stop=True, timeout=timeout, long_polling_timeout=long_polling_timeout,
  File "/usr/local/lib/python3.10/dist-packages/telebot/__init__.py", line 1043, in polling
    self.__threaded_polling(non_stop=non_stop, interval=interval, timeout=timeout, long_polling_timeout=long_polling_timeout,
  File "/usr/local/lib/python3.10/dist-packages/telebot/__init__.py", line 1118, in __threaded_polling
    raise e
  File "/usr/local/lib/python3.10/dist-packages/telebot/__init__.py", line 1074, in __threaded_pol